This first cell downloads the dataset, skip if you already have it.

In [1]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

--2020-11-03 17:14:07--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.206.144, 2a00:1450:4009:811::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.206.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  44.5MB/s    in 5.5s    

2020-11-03 17:14:14 (42.8 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [1]:
!ls -hault AOI_5_Khartoum_Train/tfrecords/ | wc

   1015    9128   74982


In [1]:
import gdal
import sys
import ogr
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import logging
tf.get_logger().setLevel(logging.ERROR)
import random
import glob
import re
import json
%matplotlib notebook

Instructions for updating:
non-resource variables are not supported in the long term


Functions to encode the pixel data and labels.

In [9]:
def _int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _encode_sample(name, pixels, labels):
    bands, rows, cols = pixels.shape
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'name': _bytes(tf.compat.as_bytes(name)),
                'height': _int64(rows),
                'width': _int64(cols),
                'depth': _int64(bands),
                'image': _bytes(pixels.astype(np.uint8).tobytes()),
                'label': _bytes(labels.astype(np.uint8).tobytes()),
            }))

    return example

In [10]:
def rasterise(rows, cols, gt, dtype, src):
    driver = gdal.GetDriverByName('MEM')
    mem_raster = driver.Create(
        '',
        cols,
        rows,
        dtype
    )
    mem_raster.SetGeoTransform(gt)
    gdal.RasterizeLayer(mem_raster, [1], src.GetLayer(), burn_values=[1])
    
    return mem_raster

Match the tif files with their geometries.

In [11]:
tifs = glob.glob('AOI_5_Khartoum_Train/RGB-PanSharpen/*.tif')
geojsons = glob.glob('AOI_5_Khartoum_Train/geojson/buildings/*.geojson')

In [12]:
tif_ids = [int(re.findall(r'\d+', tif)[2]) for tif in tifs]
geojson_ids = [int(re.findall(r'\d+', js)[2]) for js in geojsons]

train_data = [[tifs[i], geojsons[geojson_ids.index(id_)]] for i, id_ in enumerate(tif_ids)]

Loop through the files and add to tfrecords.

In [59]:
out_path = 'AOI_5_Khartoum_Train/tfrecords/'
fn_root = 'aoi5_khartoum_train_'

track_sum = np.array([0., 0., 0.])

def scale_to_8bit(pix):
    scaled = np.zeros(pix.shape, dtype=float)
    for b in range(len(pix)):
        scaled[b] = pix[b]/pix[b].max()
    
    return (scaled*255).astype(np.uint8)

for key, (rgb_fn, labels_fn) in enumerate(train_data):
    img = gdal.Open(rgb_fn)
    gt = img.GetGeoTransform()
    nrows = img.RasterYSize
    ncols = img.RasterXSize
    pixels = img.ReadAsArray()
    
    # Scale to 8bit
    nbands, nrows, ncols = pixels.shape
    pix_8bit = scale_to_8bit(pixels)
    
    band_mean = pix_8bit.reshape((3, nrows*ncols)).sum(axis=1)
    track_sum += band_mean

    geo = ogr.Open(labels_fn)
    layer = geo.GetLayer()
    
    labels = rasterise(nrows, ncols, gt, gdal.GDT_Byte, geo).ReadAsArray()
    record = _encode_sample(str(tif_ids[key]), pix_8bit, labels)
    writer = tf.io.TFRecordWriter(
            os.path.join(out_path,'{}{:04d}.tfrecords'.format(fn_root, tif_ids[key]))
        )
    writer.write(record.SerializeToString())
    geo = None
    img = None
    writer.close()
    